# TD3: AES algorithm

The  Advanced  Encryption  Standard  (AES)  specifies  a  FIPS-approved  cryptographic  algorithm  that  can  be  used  to  protect  electronic  data.    The  AES  algorithm  is  a  symmetric   block   cipher   that   can   encrypt   (encipher)   and   decrypt   (decipher)   information.   Encryption  converts  data  to  an  unintelligible  form  called  ciphertext;    decrypting  the  ciphertext converts the data back into its original form, called plaintext.
All the specification is available at: https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.197.pdf

# PART A: encryption

# Round-keys generation

The code below generates all keys for encryption and decryption.

In [ ]:
Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)


def text2matrix(text):
    matrix = []
    for i in range(16):
        byte = (text >> (8 * (15 - i))) & 0xFF
        if i % 4 == 0:
            matrix.append([byte])
        else:
            matrix[int(i / 4)].append(byte)
    return matrix

def change_key(master_key):
    round_keys = text2matrix(master_key)
    print ('\n original key : \n', round_keys)
    for i in range(4, 4 * 11):
        round_keys.append([])
        if i % 4 == 0:
            byte = round_keys[i - 4][0]        \
                     ^ Sbox[round_keys[i - 1][1]]  \
                     ^ Rcon[int(i / 4)]
            round_keys[i].append(byte)

            for j in range(1, 4):
                    byte = round_keys[i - 4][j]    \
                         ^ Sbox[round_keys[i - 1][(j + 1) % 4]]
                    round_keys[i].append(byte)
        else:
            for j in range(4):
                    byte = round_keys[i - 4][j]    \
                         ^ round_keys[i - 1][j]
                    round_keys[i].append(byte)

    print ('\n cipher key : \n', round_keys)
    return(round_keys)

Understand the proposed code and compare it with key generation of AES standart.

**QUESTION:** Try different master keys and check the generated cypher keys for each round.

In [2]:
master_key = 0x2b7e151628aed2a6abf7158809cf4f3c
print('\n master key : \n', hex(master_key))

round_keys=change_key(master_key)


 master key : 
 0x2b7e151628aed2a6abf7158809cf4f3c

 original key : 
 [[43, 126, 21, 22], [40, 174, 210, 166], [171, 247, 21, 136], [9, 207, 79, 60]]

 cipher key : 
 [[43, 126, 21, 22], [40, 174, 210, 166], [171, 247, 21, 136], [9, 207, 79, 60], [160, 250, 254, 23], [136, 84, 44, 177], [35, 163, 57, 57], [42, 108, 118, 5], [242, 194, 149, 242], [122, 150, 185, 67], [89, 53, 128, 122], [115, 89, 246, 127], [61, 128, 71, 125], [71, 22, 254, 62], [30, 35, 126, 68], [109, 122, 136, 59], [239, 68, 165, 65], [168, 82, 91, 127], [182, 113, 37, 59], [219, 11, 173, 0], [212, 209, 198, 248], [124, 131, 157, 135], [202, 242, 184, 188], [17, 249, 21, 188], [109, 136, 163, 122], [17, 11, 62, 253], [219, 249, 134, 65], [202, 0, 147, 253], [78, 84, 247, 14], [95, 95, 201, 243], [132, 166, 79, 178], [78, 166, 220, 79], [234, 210, 115, 33], [181, 141, 186, 210], [49, 43, 245, 96], [127, 141, 41, 47], [172, 119, 102, 243], [25, 250, 220, 33], [40, 209, 41, 65], [87, 92, 0, 110], [208, 20, 249, 168], [

### Other master keys generated

In [3]:
import os

#16-byte (128-bit) random master key
master_key2 = str(os.urandom(16).hex())

print('\n master key : \n', master_key2)
round_k=change_key(int(master_key2, 16))


 master key : 
 d283bcb1fa7688fe02f7eb32b54da8f4

 original key : 
 [[210, 131, 188, 177], [250, 118, 136, 254], [2, 247, 235, 50], [181, 77, 168, 244]]

 cipher key : 
 [[210, 131, 188, 177], [250, 118, 136, 254], [2, 247, 235, 50], [181, 77, 168, 244], [48, 65, 3, 100], [202, 55, 139, 154], [200, 192, 96, 168], [125, 141, 200, 92], [111, 169, 73, 155], [165, 158, 194, 1], [109, 94, 162, 169], [16, 211, 106, 245], [13, 171, 175, 81], [168, 53, 109, 80], [197, 107, 207, 249], [213, 184, 165, 12], [105, 173, 81, 82], [193, 152, 60, 2], [4, 243, 243, 251], [209, 75, 86, 247], [202, 28, 57, 108], [11, 132, 5, 110], [15, 119, 246, 149], [222, 60, 160, 98], [1, 252, 147, 113], [10, 120, 150, 31], [5, 15, 96, 138], [219, 51, 192, 232], [130, 70, 8, 200], [136, 62, 158, 215], [141, 49, 254, 93], [86, 2, 62, 181], [117, 244, 221, 121], [253, 202, 67, 174], [112, 251, 189, 243], [38, 249, 131, 70], [247, 24, 135, 142], [10, 210, 196, 32], [122, 41, 121, 211], [92, 208, 250, 149], [177, 53, 173

In [4]:
import os

#16-byte (128-bit) random master key
master_key2 = str(os.urandom(16).hex())

print('\n master key : \n', master_key2)
round_ks=change_key(int(master_key2, 16))


 master key : 
 fc1acc96ea0547632d4c256c9b443fbc

 original key : 
 [[252, 26, 204, 150], [234, 5, 71, 99], [45, 76, 37, 108], [155, 68, 63, 188]]

 cipher key : 
 [[252, 26, 204, 150], [234, 5, 71, 99], [45, 76, 37, 108], [155, 68, 63, 188], [230, 111, 169, 130], [12, 106, 238, 225], [33, 38, 203, 141], [186, 98, 244, 49], [78, 208, 110, 118], [66, 186, 128, 151], [99, 156, 75, 26], [217, 254, 191, 43], [241, 216, 159, 67], [179, 98, 31, 212], [208, 254, 84, 206], [9, 0, 235, 229], [154, 49, 70, 66], [41, 83, 89, 150], [249, 173, 13, 88], [240, 173, 230, 189], [31, 191, 60, 206], [54, 236, 101, 88], [207, 65, 104, 0], [63, 236, 142, 189], [241, 166, 70, 187], [199, 74, 35, 227], [8, 11, 75, 227], [55, 231, 197, 94], [37, 0, 30, 33], [226, 74, 61, 194], [234, 65, 118, 33], [221, 166, 179, 127], [129, 109, 204, 224], [99, 39, 241, 34], [137, 102, 135, 3], [84, 192, 52, 124], [32, 117, 220, 192], [67, 82, 45, 226], [202, 52, 170, 225], [158, 244, 158, 157], [169, 126, 130, 203], [234, 4

**QUESTION:** write a function that prints each round key in hexadecimal, is the number of keys correct?

In [5]:
def convert2hex(round_keys):
    print('The cipher key:\n')
    for i in range(4):
        for j in range(4):
            print('\t',hex(round_keys[j][i]), end="")
        print('')

    for k in range(1, int(len(round_keys)/4)):
        print('\nRound key', k)
        for j in range(4):
            for i in range(k*4, k*4 + 4):
                print('\t',hex(round_keys[i][j]), end="")
            print('')

round_keys_hex = convert2hex(round_keys)

The cipher key:

	 0x2b	 0x28	 0xab	 0x9
	 0x7e	 0xae	 0xf7	 0xcf
	 0x15	 0xd2	 0x15	 0x4f
	 0x16	 0xa6	 0x88	 0x3c

Round key 1
	 0xa0	 0x88	 0x23	 0x2a
	 0xfa	 0x54	 0xa3	 0x6c
	 0xfe	 0x2c	 0x39	 0x76
	 0x17	 0xb1	 0x39	 0x5

Round key 2
	 0xf2	 0x7a	 0x59	 0x73
	 0xc2	 0x96	 0x35	 0x59
	 0x95	 0xb9	 0x80	 0xf6
	 0xf2	 0x43	 0x7a	 0x7f

Round key 3
	 0x3d	 0x47	 0x1e	 0x6d
	 0x80	 0x16	 0x23	 0x7a
	 0x47	 0xfe	 0x7e	 0x88
	 0x7d	 0x3e	 0x44	 0x3b

Round key 4
	 0xef	 0xa8	 0xb6	 0xdb
	 0x44	 0x52	 0x71	 0xb
	 0xa5	 0x5b	 0x25	 0xad
	 0x41	 0x7f	 0x3b	 0x0

Round key 5
	 0xd4	 0x7c	 0xca	 0x11
	 0xd1	 0x83	 0xf2	 0xf9
	 0xc6	 0x9d	 0xb8	 0x15
	 0xf8	 0x87	 0xbc	 0xbc

Round key 6
	 0x6d	 0x11	 0xdb	 0xca
	 0x88	 0xb	 0xf9	 0x0
	 0xa3	 0x3e	 0x86	 0x93
	 0x7a	 0xfd	 0x41	 0xfd

Round key 7
	 0x4e	 0x5f	 0x84	 0x4e
	 0x54	 0x5f	 0xa6	 0xa6
	 0xf7	 0xc9	 0x4f	 0xdc
	 0xe	 0xf3	 0xb2	 0x4f

Round key 8
	 0xea	 0xb5	 0x31	 0x7f
	 0xd2	 0x8d	 0x2b	 0x8d
	 0x73	 0xba	 0xf5	 0x29
	 0x21	 0xd

# Encryption

**QUESTION:** complete the three missing functions.

In [ ]:
# learnt from http://cs.ucsb.edu/~koc/cs178/projects/JT/aes.c
xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)


def matrix2text(matrix):
    text = 0
    for i in range(4):
        for j in range(4):
            text |= (matrix[i][j] << (120 - 8 * (4 * i + j)))
    return text

def encrypt(plaintext):
        plain_state = text2matrix(plaintext)

        add_round_key(plain_state, round_keys[:4])

        for i in range(1, 10):
            round_encrypt(plain_state, round_keys[4 * i : 4 * (i + 1)])

        sub_bytes(plain_state)
        shift_rows(plain_state)
        add_round_key(plain_state, round_keys[40:])

        return matrix2text(plain_state)

def add_round_key(s, k):
    for i in range(4):
        for j in range(4):
            s[i][j] ^= k[i][j]


def round_encrypt(state_matrix, key_matrix):
    "call to the four round operations"
    sub_bytes(state_matrix)
    shift_rows(state_matrix)
    mix_columns(state_matrix)
    add_round_key(state_matrix, key_matrix)


def sub_bytes(s):
    "build the sub_bytes transformation"
    row = len(s)
    col = len(s[0])
    for i in range(row):
        for j in range(col):
            s[i][j] = Sbox[s[i][j]]
    return s

def shift_rows(s):
    s[0][1],s[1][1],s[2][1],s[3][1]=s[1][1],s[2][1],s[3][1],s[0][1]
    s[0][2],s[1][2],s[2][2],s[3][2]=s[2][2],s[3][2],s[0][2],s[1][2]
    s[0][3],s[1][3],s[2][3],s[3][3]=s[3][3],s[0][3],s[1][3],s[2][3]
    return s


def mix_single_column(a):
        # please see Sec 4.1.2 in The Design of Rijndael
        t = a[0] ^ a[1] ^ a[2] ^ a[3]
        u = a[0]
        a[0] ^= t ^ xtime(a[0] ^ a[1])
        a[1] ^= t ^ xtime(a[1] ^ a[2])
        a[2] ^= t ^ xtime(a[2] ^ a[3])
        a[3] ^= t ^ xtime(a[3] ^ u)


def mix_columns(s):
        for i in range(4):
            mix_single_column(s[i])

**QUESTION:** Once the three missing functions are completed you should have a correct cipher.

In [7]:
plaintext = 0x3243f6a8885a308d313198a2e0370734
print('\n plaintext is:',hex(plaintext))

ciphertext = encrypted = encrypt(plaintext)
print('\n ciphertext is:',hex(ciphertext))
if(ciphertext == 0x3925841d02dc09fbdc118597196a0b32):
    print('\n ciphering has been done correctly')
else:
    print('\n ciphering has a problem')


 plaintext is: 0x3243f6a8885a308d313198a2e0370734

 ciphertext is: 0x3925841d02dc09fbdc118597196a0b32

 ciphering has been done correctly


# PART B: decryption

**QUESTION:** based on what you have donne for encryption complete the three missing functions.

In [ ]:
InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)


def decrypt(ciphertext):
        cipher_state = text2matrix(ciphertext)

        add_round_key(cipher_state, round_keys[40:])
        inv_shift_rows(cipher_state)
        inv_sub_bytes(cipher_state)

        for i in range(9, 0, -1):
            round_decrypt(cipher_state, round_keys[4 * i : 4 * (i + 1)])

        add_round_key(cipher_state, round_keys[:4])

        return matrix2text(cipher_state)


def round_decrypt(state_matrix, key_matrix):
    add_round_key(state_matrix, key_matrix)
    inv_mix_columns(state_matrix)
    j = inv_shift_rows(state_matrix)
    inv_sub_bytes(state_matrix)
    pass

def inv_sub_bytes(s):
    row = len(s)
    col = len(s[0])
    for i in range(row):
        for j in range(col):
            s[i][j] = InvSbox[s[i][j]]
    return s


def inv_shift_rows(s):
    s[0][1],s[1][1], s[2][1], s[3][1] = s[3][1], s[0][1], s[1][1], s[2][1]
    s[0][2],s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    s[0][3],s[1][3], s[2][3], s[3][3] = s[1][3], s[2][3], s[3][3], s[0][3]
    return s

def inv_mix_columns(s):
        # see Sec 4.1.3 in The Design of Rijndael
        for i in range(4):
            u = xtime(xtime(s[i][0] ^ s[i][2]))
            v = xtime(xtime(s[i][1] ^ s[i][3]))
            s[i][0] ^= u
            s[i][1] ^= v
            s[i][2] ^= u
            s[i][3] ^= v

        mix_columns(s)

**QUESTION:** Once the three missing functions are completed you should have a correct deciphering.

In [9]:
decrypted = decrypt(ciphertext)
print('plaintext was:', hex(plaintext))

print('decryption returns:', hex(decrypted))

plaintext was: 0x3243f6a8885a308d313198a2e0370734
decryption returns: 0x3243f6a8885a308d313198a2e0370734


# PART C: Using a library

Have a look a the crypto library from [`PyCryptodome`](https://pycryptodome.readthedocs.io/en/latest/src/introduction.html)

In [10]:
pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


**QUESTION:** Execute the following code.

In [11]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = get_random_bytes(16)
print('data is:')
for i in range(0,16):
    print(hex(data[i]))
print('key is:')
key = get_random_bytes(16)
for i in range(0,16):
    print(hex(key[i]))
cipher = AES.new(key, AES.MODE_ECB)
ciphertext = cipher.encrypt(data)

print('ciphetext is:')
for i in range(0,16):
    print(hex(ciphertext[i]))


data is:
0x8a
0x6d
0x4f
0xa1
0xfb
0xa8
0xf1
0x18
0x80
0x4c
0x49
0xa2
0x1e
0x4b
0x7
0x31
key is:
0xac
0xe5
0xda
0x4e
0xcc
0x44
0xda
0xb7
0xd2
0x46
0x98
0xfd
0x78
0x97
0x80
0x4
ciphetext is:
0x73
0x11
0x47
0xc
0xf8
0x73
0xf6
0x33
0x1c
0xf0
0xaa
0xce
0x13
0x3c
0x50
0xc3


**QUESTION:** What can you say about this code? compare the execution with the previous code you have written. Do you obtain the same result?

In [12]:
master_key_ = 0x73e2307f5fb90c915943afa53326ebce #use the same key as the execution above

print('\n master key : \n', hex(master_key))

round_keys=change_key(master_key)

plaintext = 0xe24ed3aa8cf0e9a23882526defaaa369 #use the same plaintext as the execution above
print('\n plaintext is:',hex(plaintext))

ciphertext = encrypted_ = encrypt(plaintext)
print('\n ciphertext is:',hex(ciphertext))

decrypted = decrypt(ciphertext)
print('plaintext was:', hex(plaintext))
print('decryption returns:', hex(decrypted))


 master key : 
 0x2b7e151628aed2a6abf7158809cf4f3c

 original key : 
 [[43, 126, 21, 22], [40, 174, 210, 166], [171, 247, 21, 136], [9, 207, 79, 60]]

 cipher key : 
 [[43, 126, 21, 22], [40, 174, 210, 166], [171, 247, 21, 136], [9, 207, 79, 60], [160, 250, 254, 23], [136, 84, 44, 177], [35, 163, 57, 57], [42, 108, 118, 5], [242, 194, 149, 242], [122, 150, 185, 67], [89, 53, 128, 122], [115, 89, 246, 127], [61, 128, 71, 125], [71, 22, 254, 62], [30, 35, 126, 68], [109, 122, 136, 59], [239, 68, 165, 65], [168, 82, 91, 127], [182, 113, 37, 59], [219, 11, 173, 0], [212, 209, 198, 248], [124, 131, 157, 135], [202, 242, 184, 188], [17, 249, 21, 188], [109, 136, 163, 122], [17, 11, 62, 253], [219, 249, 134, 65], [202, 0, 147, 253], [78, 84, 247, 14], [95, 95, 201, 243], [132, 166, 79, 178], [78, 166, 220, 79], [234, 210, 115, 33], [181, 141, 186, 210], [49, 43, 245, 96], [127, 141, 41, 47], [172, 119, 102, 243], [25, 250, 220, 33], [40, 209, 41, 65], [87, 92, 0, 110], [208, 20, 249, 168], [

**QUESTION:** Try other modes of operation from the library and comment the results.

In [13]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = get_random_bytes(16)
print('data is:')
for i in range(0,16):
    print(hex(data[i]))
print('key is:')
key = get_random_bytes(16)
for i in range(0,16):
    print(hex(key[i]))
cipher = AES.new(key, AES.MODE_CBC)
ciphertext = cipher.encrypt(data)

print('ciphetext is:')
for i in range(0,16):
    print(hex(ciphertext[i]))

data is:
0x19
0xee
0x15
0xe8
0x52
0xa8
0x3
0x8c
0x13
0xa9
0x34
0xe6
0x3f
0x32
0xeb
0xf4
key is:
0xc9
0xb
0x13
0x1f
0x27
0x20
0x30
0x68
0x7a
0xc
0x40
0x57
0xa
0x18
0xb3
0x92
ciphetext is:
0x61
0xa7
0x96
0xe
0xd6
0x2b
0xb4
0xe7
0x63
0xc8
0xf
0x8
0x31
0x73
0x72
0x37


In [14]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = get_random_bytes(16)
print('data is:')
for i in range(0,16):
    print(hex(data[i]))
print('key is:')
key = get_random_bytes(16)
for i in range(0,16):
    print(hex(key[i]))
cipher = AES.new(key, AES.MODE_EAX)
ciphertext = cipher.encrypt(data)

print('ciphetext is:')
for i in range(0,16):
    print(hex(ciphertext[i]))

data is:
0x47
0xec
0x6
0x3a
0x6f
0xaf
0xb1
0x81
0xfd
0x3f
0x23
0x2
0x51
0xf8
0x59
0x57
key is:
0xb6
0xf4
0x65
0xd0
0x48
0xaa
0x46
0x6b
0x9b
0x41
0x18
0xb0
0x5f
0x14
0x9a
0xf6
ciphetext is:
0xf5
0x61
0xa7
0x14
0x8e
0x5c
0x54
0x39
0x34
0xf1
0xf9
0x6f
0x1
0x86
0xea
0x45
